In [68]:
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import glob
from keras.layers import Dense
from keras import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
vggmodel = VGG16(weights='imagenet', include_top=True)




In [48]:
def IOU(box1, box2):


    x_l = max(box1[0], box2[0]) # left overlapping
    y_u = max(box1[1], box2[1]) # up overlapping
    x_r = min(box1[2], box2[2])
    y_d = min(box1[3], box2[3])


    if x_r < x_l or y_d < y_u:
        
        # no overlapping
        return 0

    s = (x_r - x_l) * (y_d - y_u) # overlapping area

    # boxes areas
    s1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    s2 = (box2[2] - box2[0]) * (box2[3] - box2[1])

    iou = s / float(s1 + s2 - s)

    return iou

In [47]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

train_images=[]
train_labels=[]

files = glob.glob('data*.csv')
for e,i in enumerate(os.listdir(annot)):
    try:
        if i.startswith("airplane"):
            filename = i.split(".")[0]+".jpg"
            print(e,filename)
            image = cv2.imread(os.path.join(path,filename))
            df = pd.read_csv(os.path.join(annot,i))
            gtvalues=[]
            for row in df.iterrows():
                x1 = int(row[1][0].split(" ")[0])
                y1 = int(row[1][0].split(" ")[1])
                x2 = int(row[1][0].split(" ")[2])
                y2 = int(row[1][0].split(" ")[3])
                gtvalues.append({"x1":x1,"x2":x2,"y1":y1,"y2":y2})
            ss.setBaseImage(image)
            ss.switchToSelectiveSearchFast()
            ssresults = ss.process()
            imout = image.copy()
            counter = 0
            falsecounter = 0
            flag = 0
            fflag = 0
            bflag = 0
            for e,result in enumerate(ssresults):
                if e < 2000 and flag == 0:
                    for gtval in gtvalues:
                        x,y,w,h = result
                        iou = get_iou(gtval,{"x1":x,"x2":x+w,"y1":y,"y2":y+h})
                        if counter < 30:
                            if iou > 0.70:
                                timage = imout[y:y+h,x:x+w]
                                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                                train_images.append(resized)
                                train_labels.append(1)
                                counter += 1
                        else :
                            fflag =1
                        if falsecounter <30:
                            if iou < 0.3:
                                timage = imout[y:y+h,x:x+w]
                                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                                train_images.append(resized)
                                train_labels.append(0)
                                falsecounter += 1
                        else :
                            bflag = 1
                    if fflag == 1 and bflag == 1:
                        print("inside")
                        flag = 1
    except Exception as e:
        print(e)
        print("error in "+filename)
        continue

NameError: ignored

In [49]:
selective_search = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

train_images=[]
train_labels=[]


my_cols = ["x1", "y1", "x2", "y2"]

images = [cv2.imread(im) for im in sorted(glob.glob("/content/sample_data/rcnn/airplane*.jpg"))]
labels = [pd.read_csv(csv, names = my_cols, engine='python', skiprows=[0], delimiter=" ") for csv in sorted(glob.glob('/content/sample_data/rcnn/airplane*.csv'))] 



im = images[0]
df = labels[0]
print(df.shape, df)
x = np.array(df['x1'])
print(labels[0].head)
print(x)


(3, 4)     x1   y1   x2   y2
0  173   15  232   70
1   49   76  139  153
2    4  164   93  244
<bound method NDFrame.head of     x1   y1   x2   y2
0  173   15  232   70
1   49   76  139  153
2    4  164   93  244>
[173  49   4]


In [54]:
for i in range(len(images)):

  im = images[i]
  df = labels[i]
  #print(im.shape)
  x1 = df['x1']
  y1 = df['y1']
  x2 = df['x2']
  y2 = df['y2']

  selective_search.setBaseImage(im)
  selective_search.switchToSelectiveSearchFast()
  proposals = selective_search.process()
  proposals = proposals[0:max(len(proposals), 2000)]
  #print(len(proposals))
  im2 = im.copy()

  count_1 = 0
  count_0 = 0

  for prop in proposals:

    if count_1 ==30 and count_0 == 30: break

    for box in df.itertuples():
      
      box1 = [box.x1, box.y1, box.x2, box.y2]
      x,y,w,h = prop

      iou = IOU(box, prop)

      if iou > 0.7 and count_1 < 30:

        count_1 += 1

        temp_im = im2[y:y+h,x:x+w]
        im_small = cv2.resize(temp_im, (224,224), interpolation = cv2.INTER_AREA)

        train_images.append(im_small)
        train_labels.append(1)


      elif iou < 0.3 and count_0 < 30:

        count_0 += 1

        temp_im = im2[y:y+h,x:x+w]
        im_small = cv2.resize(temp_im, (224,224), interpolation = cv2.INTER_AREA)

        train_images.append(im_small)
        train_labels.append(0)



(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)


In [91]:
X_new = np.array(train_images)
y_new = np.array(train_labels)

for layers in (vggmodel.layers)[:15]:
    print(layers)
    layers.trainable = False
X = vggmodel.layers[-2].output
predictions = Dense(2, activation="softmax")(X)
model_final = Model(inputs = vggmodel.input, outputs = predictions)
opt = Adam(lr=0.0001)
model_final.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])
#model_final.summary()


idx = np.random.rand(X_new.shape[0]) < 0.9
x_train = X_new[idx]
y_train = y_new[idx].reshape(-1,1)
x_test = X_new[~idx]
y_test = y_new[~idx].reshape(-1,1)
print(y_train.shape,x_train.shape,y_test.shape,x_test.shape)

(331, 1) (331, 224, 224, 3) (32, 1) (32, 224, 224, 3)


In [92]:
trdata = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=90)
traindata = trdata.flow(x=x_train, y=y_train)
print(traindata)
tsdata = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=90)
testdata = tsdata.flow(x=x_test, y=y_test)

In [1]:
hist = model_final.fit_generator(generator= traindata, steps_per_epoch= 10, epochs= 1000, validation_data= testdata, validation_steps=2)

NameError: ignored